In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import random
import matplotlib.image as mpimg
import splitfolders
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow import keras
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
# from tensorflow.keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras import models
from tensorflow.keras.optimizers import Adam as Adam
from tensorflow.keras.optimizers import SGD as SGD
from tensorflow.keras.optimizers import RMSprop as RMSprop
from tensorflow.keras.optimizers import Adagrad as Adagrad
from tensorflow.keras.optimizers import Adadelta as Adadelta
from tensorflow.keras.optimizers import Adamax as Adamax
from tensorflow.keras.optimizers import Nadam as Nadam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
import shutil
import torch
import tensorflow_ranking as tfr
from tensorflow_ranking.python.keras.metrics import MeanAveragePrecisionMetric
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from IPython.display import SVG
from scipy import ndimage
import pathlib as Path

2022-08-08 19:21:16.302698: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
#check gpu availability
print(device_lib.list_local_devices())
# tf.test.is_gpu_available()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1816670047261293428
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10507335808
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17832557708567587722
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


2022-08-08 19:22:49.916467: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-08 19:22:49.969313: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-08-08 19:22:49.970005: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-08-08 19:22:50.775859: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-08 19:22:50.777504: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 na

In [3]:
input_data_path =r'C:/Users/conm/Desktop/Stenosis-Project/Stenosis detection/input'

train_path = '/home/lunet/conm/Desktop/Stenosis-Project/output/train/stenosis'

train_path1 = r'C:\Users\conm\Desktop\Stenosis-Project\train_images'

test_path = '/home/lunet/conm/Desktop/Stenosis-Project/test_images'

val_path = '/home/lunet/conm/Desktop/Stenosis-Project/output/val/stenosis'

train_path2 = r'C:\Users\conm\Desktop\Stenosis-Project\Dataset_Reduced\train\images'

In [4]:
# Read train_labels.csv
train_labels_csv = pd.read_csv('train_labels.csv')
train_labels_csv

,filename,width,height,class,xmin,ymin,xmax,ymax
0,14_024_2_0042.bmp,800,800,Stenosis,235,156,282,192
1,14_031_4_0031.bmp,800,800,Stenosis,419,319,446,349
2,14_046_5_0066.bmp,1000,1000,Stenosis,679,307,699,336
3,14_010_6_0035.bmp,512,512,Stenosis,94,244,126,286
4,14_014_1_0015.bmp,512,512,Stenosis,163,118,203,151
...,...,...,...,...,...,...,...,...
6737,14_066_7_0059.bmp,800,800,Stenosis,229,100,254,160
6738,14_088_8_0077.bmp,800,800,Stenosis,512,319,576,347
6739,14_095_7_0045.bmp,800,800,Stenosis,199,218,229,273
6740,14_051_6_0026.bmp,512,512,Stenosis,258,181,277,216


In [5]:
test_labels_csv = pd.read_csv('test_labels.csv')
test_labels_csv

,filename,width,height,class,xmin,ymin,xmax,ymax
0,14_029_7_0058.bmp,800,800,Stenosis,288,369,328,398
1,14_024_1_0052.bmp,800,800,Stenosis,166,160,202,202
2,14_095_2_0075.bmp,800,800,Stenosis,461,337,492,379
3,14_075_6_0035.bmp,512,512,Stenosis,128,136,189,158
4,14_081_7_0026.bmp,800,800,Stenosis,323,177,351,230
...,...,...,...,...,...,...,...,...
828,14_066_8_0028.bmp,800,800,Stenosis,125,175,161,216
829,14_048_1_0057.bmp,1000,1000,Stenosis,264,427,306,458
830,14_051_2_0051.bmp,512,512,Stenosis,82,297,110,344
831,14_051_2_0025.bmp,512,512,Stenosis,77,248,102,284


In [6]:
val_labels_csv = pd.read_csv('val_labels.csv')
val_labels_csv

,filename,width,height,class,xmin,ymin,xmax,ymax
0,14_029_5_0059.bmp,800,800,Stenosis,192,423,237,459
1,14_006_1_0138.bmp,800,800,Stenosis,471,438,560,575
2,14_087_1_0090.bmp,800,800,Stenosis,256,250,335,300
3,14_021_37_0019.bmp,512,512,Stenosis,187,408,239,431
4,14_057_3_0061.bmp,1000,1000,Stenosis,389,117,443,174
...,...,...,...,...,...,...,...,...
745,14_029_1_0068.bmp,800,800,Stenosis,297,265,356,310
746,14_095_2_0035.bmp,800,800,Stenosis,435,375,473,416
747,14_095_1_0080.bmp,800,800,Stenosis,501,498,546,529
748,14_048_3_0074.bmp,1000,1000,Stenosis,458,181,479,199


In [41]:
train_images = []
train_targets = []

In [42]:
val_images = []
val_targets = []

In [43]:
test_images = []
test_targets = []

In [12]:
for index, row in train_labels_csv.iterrows():
    (filename, width, height, class_name, xmin, ymin, xmax, ymax) = row

    pic = cv2.imread(os.path.join(train_path,filename))
    pic = cv2.resize(pic, (600, 600))
    pic_arr = keras.preprocessing.image.img_to_array(pic)

    train_images.append(pic)
    train_targets.append((xmin, ymin, xmax, ymax))

In [45]:
for index, row in val_labels_csv.iterrows():
    (filename, width, height, class_name, xmin, ymin, xmax, ymax) = row

    pic = cv2.imread(os.path.join(val_path,filename))
    pic = cv2.resize(pic, (600, 600))
    pic_arr = keras.preprocessing.image.img_to_array(pic)

    val_images.append(pic)
    val_targets.append((xmin, ymin, xmax, ymax))

In [44]:
for index, row in test_labels_csv.iterrows():
    (filename, width, height, class_name, xmin, ymin, xmax, ymax) = row

    pic = cv2.imread(os.path.join(test_path,filename))
    pic = cv2.resize(pic, (600, 600))
    pic_arr = keras.preprocessing.image.img_to_array(pic)

    test_images.append(pic)
    test_targets.append((xmin, ymin, xmax, ymax))

In [15]:
train_images = np.asarray(train_images).astype('float32')
train_targets = np.asarray(train_targets).astype('float32')

In [14]:
val_images = np.asarray(val_images).astype('float32')
val_targets = np.asarray(val_targets).astype('float32')

In [13]:
test_images = np.asarray(test_images).astype('float32')
test_targets = np.asarray(test_targets).astype('float32')

In [16]:
test_images.shape

(833, 300, 300, 3)

In [17]:
val_images.shape

(750, 300, 300, 3)

In [18]:
train_images.shape

(6742, 300, 300, 3)

In [20]:
# Normalize targets
train_targets = train_targets / 256
val_targets = val_targets / 256

# Normalize images
train_images = train_images / 255
val_images = val_images / 255

# Normalize test images
test_images = test_images / 255
test_targets = test_targets / 256

In [22]:
dataGenerator = ImageDataGenerator(rotation_range=0, 
                                   zoom_range=0, 
                                   width_shift_range=0, 
                                   height_shift_range=0, 
                                   horizontal_flip=True,
                                   vertical_flip=True)

In [40]:
# Create a new model for object detection and transfer learning
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(600, 600, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(4))
# Add custom metric for mean average precision to model.compile
# add learning rate

model.compile(loss='mse', optimizer=SGD(learning_rate=0.1), metrics=[tfr.keras.metrics.MeanAveragePrecisionMetric()])
# model.compile(loss='mse', optimizer='adam', metrics=['mae'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        3

In [38]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

In [39]:
keras.backend.clear_session()
np.random.seed(15)
tf.random.set_seed(15)
# Model training
# Add confidence threshold to filter out false positives
model.fit(train_images, train_targets, validation_data=(val_images, val_targets), epochs=100, batch_size=32, callbacks=[es])


# confidence_threshold = 0.5

model.fit(dataGenerator.flow(train_images, train_targets, batch_size=32),
                            validation_data=(val_images, val_targets), 
                            epochs=150, callbacks=es, verbose=1)

2022-08-08 21:06:46.141932: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 7281360000 exceeds 10% of free system memory.


Epoch 1/100
211/211 [==============================] - 7s 33ms/step - loss: 0.5276 - mean_average_precision_metric_1: 0.6758 - val_loss: 0.2355 - val_mean_average_precision_metric_1: 0.6964
Epoch 2/100
211/211 [==============================] - 7s 31ms/step - loss: 0.2384 - mean_average_precision_metric_1: 0.7016 - val_loss: 0.2252 - val_mean_average_precision_metric_1: 0.6996
Epoch 3/100
211/211 [==============================] - 7s 31ms/step - loss: 0.2052 - mean_average_precision_metric_1: 0.7165 - val_loss: 0.1698 - val_mean_average_precision_metric_1: 0.7346
Epoch 4/100
211/211 [==============================] - 7s 31ms/step - loss: 0.1764 - mean_average_precision_metric_1: 0.7404 - val_loss: 0.1262 - val_mean_average_precision_metric_1: 0.7608
Epoch 5/100
211/211 [==============================] - 7s 31ms/step - loss: 0.1250 - mean_average_precision_metric_1: 0.7594 - val_loss: 0.0994 - val_mean_average_precision_metric_1: 0.7671
Epoch 6/100
211/211 [=============================

KeyboardInterrupt: 

In [52]:
# test trained modeel
test_map = model.evaluate(test_images, test_targets)
test_map

27/27 [==============================] - 0s 11ms/step - loss: 0.0047 - mean_average_precision_metric_11: 0.7956


[0.004733323585242033, 0.7956182360649109]

In [ ]:
# Save model
model.save('model.h5')
